In [1]:
%pip install gmpy2 cryptography
import os
import time
import itertools
import gmpy2

from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization

from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.kdf.scrypt import Scrypt
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
import os, sys, struct

READ_SIZE = 4096

def encrypt_file(plainpath, cipherpath, password):
    
    salt = os.urandom(16)
    kdf = Scrypt(salt=salt, length=32, n=2**14, r=8, p=1,
                    backend=default_backend())
    key = kdf.derive(password)

    iv = os.urandom(12)

    encryptor = Cipher(
        algorithms.AES(key),
        modes.GCM(iv),
    backend=default_backend()).encryptor()

    associated_data = iv + salt


    encryptor.authenticate_additional_data(associated_data)
    with open(cipherpath, "wb+") as fcipher:
        fcipher.write(b"\x00"*(12+16+16))

        with open(plainpath, "rb") as fplain:
            for plaintext in iter(lambda: fplain.read(READ_SIZE), b''):
                ciphertext = encryptor.update(plaintext)
                fcipher.write(ciphertext)
            ciphertext = encryptor.finalize() 
            fcipher.write(ciphertext) 

            header = associated_data + encryptor.tag
            fcipher.seek(0,0)
            fcipher.write(header)

def decrypt_file(cipherpath, plainpath, password):
    with open(cipherpath, "rb") as fcipher:
        associated_data = fcipher.read(12+16)

        iv = associated_data[0:12]
        salt = associated_data[12:28]

        kdf = Scrypt(salt=salt, length=32, n=2**14, r=8, p=1, backend=default_backend())
        key = kdf.derive(password)

        tag = fcipher.read(16)

        decryptor = Cipher(
                algorithms.AES(key),
                modes.GCM(iv, tag),
                backend=default_backend()).decryptor()
        decryptor.authenticate_additional_data(associated_data)

        with open(plainpath, "wb+") as fplain:
            for ciphertext in iter(lambda: fcipher.read(READ_SIZE),b''):
                plaintext = decryptor.update(ciphertext)
                fplain.write(plaintext)

from cryptography.hazmat.primitives.ciphers.aead import AESGCM

key = AESGCM.generate_key(bit_length=256)
aesgcm = AESGCM(key)
nonce = os.urandom(12)  

plaintext = b"Paradoks"
associated_data = b"Additional data"

ciphertext = aesgcm.encrypt(nonce, plaintext, associated_data)
print(f"Ciphertext: {ciphertext}")

decrypted_text = aesgcm.decrypt(nonce, ciphertext, associated_data)
print(f"Decrypted Message: {decrypted_text.decode()}")

from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import hashes

private_key = rsa.generate_private_key(public_exponent=65537, key_size=2048, backend=default_backend())
public_key = private_key.public_key()

message = b"I love my cat"
ciphertext = public_key.encrypt(
    message,
    padding.OAEP(mgf=padding.MGF1(algorithm=hashes.SHA256()), algorithm=hashes.SHA256(), label=None)
)
print(f"Ciphertext: {ciphertext}")

plaintext

Note: you may need to restart the kernel to use updated packages.
Ciphertext: b'\xa0.k\xb4\xa8\xf8>\x0bp\xa3\x00\xe6\xcas\xa2\x1f\xea\x8c\x8dOke\xe6s'
Decrypted Message: Paradoks
Ciphertext: b"8\xe8\xd6H6;\x17\x05\xdb\xb9l \xfb\x9b5\xbb?\xcan>`\xa2\x1c\x8d\xaa$\x12t!\x0c\xb4U\xdf}?\x1c\x90X\xef\xe8\xc1\xaf\x84\xdb\xca+\x89\xcb\xeb\xdf\xb2L\xd3\x02\xc5\xbes\x04\x08\xaa\n.\x99\xad\xdch\xa8\xa6@*\xf9\xd1\r\xef\xb4T\xdf9(\x99\x03\xbf\xb9jJ\xb6?\xba\xaf\xd4\xa8\xbep\x91`\xb3\xac\xf5\xb4\xe3=\xac\xba#Z\xcb\xbe\xac\x1c\xdeK'\xbd~\x1e\xa2$\xa0`\x8d\x80\x9a\xcd\xd8]=\xb7\x1c\x0e\xd8g[\x9a\x17~C\xc7\xea.M\xd9\xf0r\xd2ly\x1fV\xc2.3kM6\xcf\xc23\x00(\x94\x90x}\xcd-\xe8\\{>'\xaeL\xdfH \xbah\xbcv`&Y\xb3\x06|\xbb\\.\xb8\x8dpq`{\x80\xfcB\xd80T\xcf9y\x87\xbc\xd4C\x0b\xa5|V\x05;2d%\xc5\xe1\x8fT\xb9\x08\x89\xc2\xd1T\x97\xf6\x7fI;G\x8fq\xadb\xcd\xd1\xa6\x10w\xa8\x1b\xa9\xd7\xec\x02X@^\x8b\x9d\x04\xbc\r\x11"



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\Sofia\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


b'Paradoks'

In [2]:
private_key.decrypt(
    ciphertext,
    padding.OAEP(mgf=padding.MGF1(algorithm=hashes.SHA256()), algorithm=hashes.SHA256(), label=None)
)
print(f"Decrypted Message: {plaintext.decode()}")

from cryptography.hazmat.primitives import serialization

signature = private_key.sign(
    message,
    padding.PSS(mgf=padding.MGF1(hashes.SHA256()), salt_length=padding.PSS.MAX_LENGTH),
    hashes.SHA256()
)
print(f"Signature: {signature}")

try:
    public_key.verify(
        signature,
        message,
        padding.PSS(mgf=padding.MGF1(hashes.SHA256()), salt_length=padding.PSS.MAX_LENGTH),
        hashes.SHA256()
    )
    print("Signature Verified")
except Exception as e:
    print(f"Signature Verification Failed: {e}")

from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.scrypt import Scrypt
from cryptography.hazmat.backends import default_backend
import os
from cryptography.hazmat.primitives import hashes

digest = hashes.Hash(hashes.SHA256(), backend=default_backend())
digest.update(b"Message to hash")
hash_result = digest.finalize()
print(f"SHA-256 Hash: {hash_result.hex()}")

kdf = PBKDF2HMAC(
    algorithm=hashes.SHA256(),
    length=32,
    salt=os.urandom(16),
    iterations=100000,
    backend=default_backend()
)
key = kdf.derive(b"my password")
print(f"Derived Key: {key.hex()}")

kdf = Scrypt(
    salt=os.urandom(16),
    length=32,
    n=2**14,
    r=8,
    p=1,
    backend=default_backend()
)
key = kdf.derive(b"my password")
print(f"Derived Key: {key.hex()}")

import secrets

salt = secrets.token_hex(16)  
print("Соль:", salt)

from cryptography.hazmat.primitives.ciphers.aead import AESGCM
data = b"a secret message"
aad = b"authenticated but unencrypted data"
key = AESGCM.generate_key(bit_length=128)
aesgcm = AESGCM(key)
nonce = os.urandom(12)
ct = aesgcm.encrypt(nonce, data, aad)
aesgcm.decrypt(nonce, ct, aad)

import os
from cryptography.hazmat.primitives.ciphers.aead import AESGCM
data = b"a secret message"
aad = b"authenticated but unencrypted data"
key = AESGCM.generate_key(bit_length=128)
aesgcm = AESGCM(key)
nonce = os.urandom(12)
ct = aesgcm.encrypt(nonce, data, aad)
aesgcm.decrypt(nonce, ct, aad)

import os
from cryptography.hazmat.primitives.ciphers.aead import ChaCha20Poly1305
data = b"a secret message"
aad = b"authenticated but unencrypted data"
key = ChaCha20Poly1305.generate_key()
chacha = ChaCha20Poly1305(key)
nonce = os.urandom(12)
ct = chacha.encrypt(nonce, data, aad)
chacha.decrypt(nonce, ct, aad)

Decrypted Message: Paradoks
Signature: b'G\xa2\x0c\xe9\xf6\xcd\xcd,;\xd4\xdb\x8e\xb7(\'\x0b\xeewJ6\xdeJ\x1a&\x83\x85\xd5\x0b\xdc\xc75\xef\x05\xbb\x83\x14\xdb\xd5\xd2\x08v\x14\x0fdo\xbd\x83bT\x1f\x90?``nA/u\xedS\x17\xae\x8c\x9fO+\x1a*\xd7S\xc7\xf7\xae\x89l=\xfe\x868\x84{\x98s\xe8O\xe3\x9cy\xfc.\xe41\x0eg\xe4y\x0c\x18\xbd\x91Z\x9a\\\t\x90qf\xf6p\xcaW\xf1z\x83\\\x07\x86T\xa4\x17\xee"\x1b\x99\x94\xf0{r\xdc\xd0\x9b\xb2`\xd1d@u"cxJ\x18W\x84Y \xd3\xaar\x94\xc0\x7ff%\xe1\x92B\xcb^\x9dE\xd2\xaeC\x94a\x112\xfd\x9a\x8aK\x81r\xb4\xa5\xe1\x9f\x08\x893\xb4\xcc5et#\nV\x91uD\x1fA\xa4\xce\x03\x17\x8bP\xc8\xdbb\xa3\xb7I\xa1{\xca)\xfc\xf9TN=\r_\xdb\x00\x16\x99`\xa6\xbfm\xdc^HD\x0f2\x1a\xf6\xd1LO\xf8\x8eY\x1a\x9a\x11{JS\x99lI\x8e^\'\x9e\xf0\xf6\xc9\x07'
Signature Verified
SHA-256 Hash: f1aa45b0f5f6703468f9b9bc2b9874d4fa6b001a170d0f132aa5a26d00d0c7e5
Derived Key: 3844cd7007b212f60439ba26473a042ebc48c7b8a6e8584522919258b670858d
Derived Key: c2159060d16d0ad55f67cd7c65760a321cf87b264e58ed8749919085417cb677
Со

b'a secret message'